# Đề bài

Cho một danh sách A chứa những từ thuộc ngôn ngữ tự nhiên và một số nguyên b cho biết giới hạn về số lượng ký tự có thể chứa trong một dòng của văn bản. Bài toán đặt ra là bố trí các từ một cách thẫm mỹ vào các hàng để tạo nên một văn bản hoàn chỉnh.

# Input

Một danh sách A, trong đó
- Một từ trong danh sách A không được tồn tại kí tự " "
- Dộ dài mỗi từ phải được đảm bảo là lớn hơn 0
- Đầu vào chương trình là A phải chứa ít nhất một kí tự

và các điều kiện 
- 1 <= A.length <= 300
- 1 <= A[i].length <= 300
- 1 <= maxWidght <= 100
    
# Output
nguồn: [leetcode](https://leetcode.com/problems/text-justification/)
```
Input: words = ["What","must","be","acknowledgment","shall","be"], maxWidth = 16

Output:
[
  "What   must   be",
  "acknowledgment  ",
  "shall be        "
]

Input: words = ["Science","is","what","we","understand","well","enough","to","explain","to","a","computer.","Art","is","everything","else","we","do"], maxWidth = 20

Output:
[
  "Science  is  what we",
  "understand      well",
  "enough to explain to",
  "a  computer.  Art is",
  "everything  else  we",
  "do                  "
]

Input: words = ["This", "is", "an", "example", "of", "text", "justification."], maxWidth = 16

Output:
[
   "This    is    an",
   "example  of text",
   "justification.  "
]
```


# Thiết kế thuật toán
**1. Ý TƯỞNG:**

In [8]:
import re

input_ = """
Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam
voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet
clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit
amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam
nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed
diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet
clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.
"""

words = re.findall(r"\w+",input_)
print(words)

['Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'consetetur', 'sadipscing', 'elitr', 'sed', 'diam', 'nonumy', 'eirmod', 'tempor', 'invidunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliquyam', 'erat', 'sed', 'diam', 'voluptua', 'At', 'vero', 'eos', 'et', 'accusam', 'et', 'justo', 'duo', 'dolores', 'et', 'ea', 'rebum', 'Stet', 'clita', 'kasd', 'gubergren', 'no', 'sea', 'takimata', 'sanctus', 'est', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'consetetur', 'sadipscing', 'elitr', 'sed', 'diam', 'nonumy', 'eirmod', 'tempor', 'invidunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliquyam', 'erat', 'sed', 'diam', 'voluptua', 'At', 'vero', 'eos', 'et', 'accusam', 'et', 'justo', 'duo', 'dolores', 'et', 'ea', 'rebum', 'Stet', 'clita', 'kasd', 'gubergren', 'no', 'sea', 'takimata', 'sanctus', 'est', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet']


Trong quá khứ, microsoft đã tiếp cận vấn đề bằng phương pháp tham lam nhưng phương pháp này được nhận định là không hiệu quả.

Với hướng tiếp cận tham lam, việc căn chỉnh văn phải được định nghĩa là phải chứa tất cả các từ vào một dòng cho đến khi đạt đến giới hạn b. Tuy nhiên, với hướng tiếp cận tham lam như thế thì sẽ dễ dàng sảy ra trường hợp như sau:

Giả sử như ta có một dãy kí tự A = ["blah", "blah", "blah", "blah", "somethinglongs"] với b = 14.
Khi căn chỉnh với ý tưởng của phương pháp tham lam thì ta có:

```
blah blah blah
blah
somethinglongs
```

nhưng thứ chúng ta muốn đạt được là:

```
blah      blah
blah      blah
somethinglongs
```

In [31]:
#Tiếp cận tham lam
space = textwidth - len(words[0])
print(words[0], end='')
for x in range(1, len(words)):
    l = len(words[x])
    if l <= space:
        space -= l+1
        print(" " + words[x], end='')
    else:
        print("\n" + words[x], end='')
        space = textwidth - l

Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy eirmod
tempor invidunt ut labore et dolore magna aliquyam erat sed diam voluptua At vero
eos et accusam et justo duo dolores et ea rebum Stet clita kasd gubergren no sea
takimata sanctus est Lorem ipsum dolor sit amet Lorem ipsum dolor sit amet
consetetur sadipscing elitr sed diam nonumy eirmod tempor invidunt ut labore et
dolore magna aliquyam erat sed diam voluptua At vero eos et accusam et justo duo
dolores et ea rebum Stet clita kasd gubergren no sea takimata sanctus est Lorem
ipsum dolor sit amet

**2. THUẬT TOÁN:**

Nhận thấy được sự hạn chế này, LaTex đã tiếng cận vấn đề này theo một hướng huy hoạch động.

Quy hoạch động về nội tại là một phương pháp thiết kế giải thuật với mục tiêu chính là hi sinh sự độ phức tạp về không gian nhưng đổi lại có thể cải thiện độ phức tạp về thời gian(tuy nhiên, số mức độ hi sinh của không gian không đáng kể so với thời gian, nên quy hoạch động là một phương pháp cho ra một hướng giải quyết một bài toán theo hướng chung(general) đáng để sử dụng nếu được cài đặt một cách khôn ngoan). Chúng ta có thể lưu trử giá trị mỗi lần tính toán và gọi lại thay vì phải tái tính toán cho bước tính toán đó lần nữa.

Trong LaTex, một hướng tiếp cận được đặt ra là tìm ra một công thức để biểu diễn độ thẩm mỹ hoặc ngược lại là độ mất thẩm mỹ của một dòng được tạo ra bằng cách lấy ra các từ trong danh sách A.

Badness(lineLen, max) -> { (max - lineLen)^3 && if max - lineLen < 0 then `infinite`

Vấn đề về text justification CÓ THỂ THAM KHẢO Ở: [here](https://www.youtube.com/watch?v=ENyox7kNKeY&t=1260s)

In [2]:
def badness(line,textwidth):

    # Number of gaps
    length_line = len(line) - 1

    for word in line:
        length_line += len(word)

    if length_line > textwidth: return float('inf')

    return ( textwidth - length_line )**3

In [3]:
import numpy as np

textwidth = 80

DP = [0]*(len(words)+1)

for i in range(len(words)-1,-1,-1):
    DP[i] = np.min([DP[j] + badness(words[i:j],textwidth) for j in range(i+1,len(words)+1)])

In [4]:
DP = [0]*(len(words)+1)
breaks = [0]*(len(words)+1)

for i in range(len(words)-1,-1,-1):
    temp = [DP[j] + badness(words[i:j], textwidth) for j in range(i+1,len(words)+1)]

    index = np.argmin(temp)

    # Index plus position in upper list
    breaks[i] = index + i + 1
    DP[i] = temp[index]

In [14]:
def reconstruct_text(words,breaks):                                                                                                                

    lines = []
    linebreaks = []

    i = 0 
    while True:

        linebreaks.append(breaks[i])
        i = breaks[i]

        if i == len(words):
            linebreaks.append(0)
            break

    for i in range( len(linebreaks) ):
        if len(words[ linebreaks[i-1] : linebreaks[i] ]) == 0:
            continue
        lines.append( ' '.join( words[ linebreaks[i-1] : linebreaks[i] ] ).strip() )

    return lines

In [20]:
def spacing(text,textwidth,maxspace=4):

    for i in range(len(text)):

        length_line = len(text[i])

        if length_line < textwidth:

            status_length = length_line
            whitespaces_remain = textwidth - status_length
            Nwhitespaces = text[i].count(' ')

            # If whitespaces (to add) per whitespace exeeds
            # maxspace, don't do anything.
            if int(whitespaces_remain/Nwhitespaces) > maxspace - 1:
                pass
            else:
                text[i] = text[i].replace(' ',' '*( 1 + int(whitespaces_remain/Nwhitespaces)) )
                status_length = len(text[i])

                # Periods have highest priority for whitespace insertion
                periods = text[i].split('.')

                # Can we add a whitespace behind each period?
                if len(periods) - 1 + status_length <= textwidth:
                    text[i] = '. '.join(periods).strip()

                status_length = len(text[i])
                whitespaces_remain = textwidth - status_length
                Nwords = len(text[i].split())
                Ngaps = Nwords - 1

                if whitespaces_remain != 0:factor = Ngaps / whitespaces_remain

                # List of whitespaces in line i
                gaps = re.findall('\s+', text[i])

                temp = text[i].split()
                for k in range(Ngaps):
                    temp[k] = ''.join([temp[k],gaps[k]])

                for j in range(whitespaces_remain):
                    if status_length >= textwidth:pass
                    else:
                        replace = temp[int(factor*j)]
                        replace = ''.join([replace, " "])
                        temp[int(factor*j)] = replace
                text[i] = ''.join(temp)

    return text

In [21]:
text = reconstruct_text(words,breaks)
for line in text:
    print(line)

Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat sed diam
voluptua At vero eos et accusam et justo duo dolores et ea rebum Stet
clita kasd gubergren no sea takimata sanctus est Lorem ipsum dolor sit
amet Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat sed diam
voluptua At vero eos et accusam et justo duo dolores et ea rebum Stet
clita kasd gubergren no sea takimata sanctus est Lorem ipsum dolor sit amet


In [23]:
text = spacing(text,textwidth)
for line in text:
    print(line)

Lorem  ipsum  dolor  sit  amet  consetetur  sadipscing  elitr  sed  diam  nonumy
eirmod  tempor  invidunt  ut  labore  et dolore  magna  aliquyam  erat  sed diam
voluptua  At  vero  eos  et  accusam  et justo  duo  dolores  et  ea  rebum Stet
clita  kasd  gubergren  no  sea  takimata  sanctus  est  Lorem  ipsum  dolor sit
amet  Lorem ipsum  dolor sit  amet consetetur  sadipscing elitr  sed diam nonumy
eirmod  tempor  invidunt  ut  labore  et dolore  magna  aliquyam  erat  sed diam
voluptua  At  vero  eos  et  accusam  et justo  duo  dolores  et  ea  rebum Stet
clita  kasd gubergren  no sea  takimata sanctus est  Lorem ipsum  dolor sit amet
